# Exploring the Arts Council National Portfolio 2018-2022

On 27 June 2017, the [Arts Council for England](http://www.artscouncil.org.uk) announced the [National Portfolio Organisations](http://www.artscouncil.org.uk/NPO) for the next four years, a [factsheet](http://www.artscouncil.org.uk/sites/default/files/download-file/Investment__factsheet.pdf) and a [dataset](http://www.artscouncil.org.uk/sites/default/files/download-file/NPO_successful_20182022_July_6.xlsx)! 

This prompted a few questions:

* Is the dataset open data?
* Can the dataset be improved in any way?

**Author**: Edafe Onerhime

**Created**: 2017-07-18

**Last Updated**: 2017-08-04

**Description**:  Explores the Arts Council National Portfolio 2018-2022 dataset

**Contents**: Notebook folder with this Jupyter Notebook, data folder, research folder

## Q1. Is the dataset open data?

According to the Arts Council's [Terms and conditions](http://www.artscouncil.org.uk/terms-and-conditions#section-3) it is open data!

> The copyright in the contents of this website is owned by Arts Council England or its licensors.  The data and information available through www.artscouncil.org.uk are owned by the Arts Council England and unless otherwise noted this is operated under the Open Government Licence.

> Please contact copyright@artscouncil.org.uk if you wish to use any of the materials on this website

## Q2. Can the dataset be improved?

Yes, we can make it simpler for analysis,let's follow some steps from Hadley Wickham's [Tidy Data](http://vita.had.co.nz/papers/tidy-data.pdf)


In [1]:
import os.path
import pandas as pd
import numpy as np
from urllib.request import urlretrieve

In [2]:
fileDetails = {'NPO_successful_20182022_July_6.xlsx': 'http://www.artscouncil.org.uk/sites/default/files/download-file/NPO_successful_20182022_July_6.xlsx'}

dataInputFolder = os.path.join(os.pardir,'data','input')
dataOutputFolder = os.path.join(os.pardir,'data','output')

outputCSVAll = os.path.join(dataOutputFolder,'NPO_successful_20182022_July_6_all.csv')
outputCSVPortfolio = os.path.join(dataOutputFolder,'NPO_successful_20182022_July_6_portfolio.csv')
outputCSVFunding = os.path.join(dataOutputFolder,'NPO_successful_20182022_July_6_funding.csv')
outputXLS = os.path.join(dataOutputFolder,'NPO_successful_20182022_July_6_all.xlsx')

In [3]:
def portFolioFunding(dfIn, ColumnName, Year, Period, Type):
    """ Returns a dataframe with Portfolio Funding amount, Year e.g. 2015 and Period e.g. 2015-2018"""
    dfOut = dfIn[[ColumnName]].copy()
    dfOut['Year'] = Year
    dfOut['Period'] = Period
    dfOut['Type'] = Type
    dfOut.rename(columns={ColumnName:'Amount'}, inplace=True)
    return dfOut

In [4]:
# Get the file
for fname in fileDetails:
    print('Checking for:',fname)
    fname = os.path.join(dataInputFolder,fname)   
    if not os.path.isfile(fname):
        print('Downloading:',fname)
        urlretrieve(fileDetails[fname], fname)

Checking for: NPO_successful_20182022_July_6.xlsx


In [5]:
# Load into Pandas
fname = os.path.join(dataInputFolder,'NPO_successful_20182022_July_6.xlsx')   
df = pd.read_excel(open(fname,'rb'), sheetname=0)

In [6]:
# Tidy up - no major changes

# Strip out carriage returns in column names
df = df.rename(columns={col: col.replace('\n','').replace('  ',' ') for col in df.columns})

# Strip out text value 'NEW' so column is all numbers
df.loc[df['% Cash change between 17/18 and 18/19'].str.contains('NEW', na=False), '% Cash change between 17/18 and 18/19'] = np.NaN

# Make these columns categories
df['Funding Band'] = df['Funding Band'].apply(str).astype('category')
df['Area'] = df['Area'].astype('category')
df['ACE Region'] = df['ACE Region'].astype('category')
df['ONS Region'] = df['ONS Region'].astype('category')
df['Discipline'] = df['Discipline'].astype('category')
df['Funding Source (GIA or LOT)'] = df['Funding Source (GIA or LOT)'].astype('category')

# Make these columns true or false (Bool)
df.loc[df['Portfolio funded in 2015-18?'].str.contains('Yes', na=False), 'Portfolio funded in 2015-18?'] = True
df.loc[df['Portfolio funded in 2015-18?'].str.contains('No', na=False), 'Portfolio funded in 2015-18?'] = False
df['Portfolio funded in 2015-18?'] = df['Portfolio funded in 2015-18?'].astype('bool')

# Don't care about the pennies - make the columns ints
df['Portfolio funding 15/16 - £'] = df['Portfolio funding 15/16 - £'].astype('int')
df['Portfolio funding 16/17 - £'] = df['Portfolio funding 16/17 - £'].astype('int')
df['Portfolio funding 17/18 - £'] = df['Portfolio funding 17/18 - £'].astype('int')
df['TOTAL Portfolio funding 15/18 - £'] = df['TOTAL Portfolio funding 15/18 - £'].astype('int')

In [7]:
# Enrichment - New columns mostly

# Enrich: Funding Source (GIA or LOT) using descriptions here: https://www.artsprofessional.co.uk/news/npo-scheme-merge-ace-grant-aid-and-lottery-funding
df['Funding Source Description'] = df['Funding Source (GIA or LOT)'].astype(str)
df.loc[df['Funding Source Description'].str.contains('GIA', na=False), 'Funding Source Description'] = 'Grant in Aid'
df.loc[df['Funding Source Description'].str.contains('LOT', na=False), 'Funding Source Description'] = 'National Lottery'

# Enrich: Funding Bands using descriptions here: http://www.artscouncil.org.uk/sites/default/files/download-file/Briefing_current_NPOs_MPMs_FINAL.pptx
df['Funding Band Description'] = df['Funding Band'].astype(str)
df.loc[~df['Funding Band Description'].str.contains('SSO'), 'Funding Band Description'] = 'Band ' + df['Funding Band Description'].astype(str)
df.loc[df['Funding Band Description'].str.contains('SSO'), 'Funding Band Description'] = 'Sector Support Organisation'

# Pull information out of notes: Bridge Organisation, Uplift, Reduction, Museum Development
df['Bridge Organisation'] = False
df['Technical Uplift'] = False
df['Technical Reduction'] = False
df['Museum Development'] = False
df.loc[df['Notes'].str.lower().str.contains('bridge organisation', na=False), 'Bridge Organisation'] = True
df.loc[df['Notes'].str.lower().str.contains('uplift', na=False), 'Technical Uplift'] = True
df.loc[df['Notes'].str.lower().str.contains('reduction', na=False), 'Technical Reduction'] = True
df.loc[df['Notes'].str.lower().str.contains('museum development', na=False), 'Museum Development'] = True


In [8]:
# Add Index (Looks like Applicant Name & Funding Band are unique)
df.set_index(['Applicant Name', 'Funding Band'], inplace=True)

In [9]:
# Normalise by splitting funidng from portfolio (naming mine!)

# Stack the funding
dfFunding = portFolioFunding(df,'Portfolio funding 15/16 - £','2015','2015-2018','Fund')
dfFunding = dfFunding.append(portFolioFunding(df,'Portfolio funding 16/17 - £','2016','2015-2018','Fund'))
dfFunding = dfFunding.append(portFolioFunding(df,'Portfolio funding 17/18 - £','2017','2015-2018','Fund'))
dfFunding = dfFunding.append(portFolioFunding(df,'Portfolio grant 18/19 - £','2018','2018-2022','Grant'))
dfFunding = dfFunding.append(portFolioFunding(df,'Portfolio grant 19/20 - £','2019','2018-2022','Grant'))
dfFunding = dfFunding.append(portFolioFunding(df,'Portfolio grant 20/21 - £','2020','2018-2022','Grant'))
dfFunding = dfFunding.append(portFolioFunding(df,'Portfolio grant 21/22 - £','2021','2018-2022','Grant'))

# Add Year to index
dfFunding.set_index('Year', append=True, inplace=True)
dfFunding['Period'] = dfFunding['Period'].astype('category')
dfFunding['Type'] = dfFunding['Type'].astype('category')

# Create seperate portfolio
dfPortfolio = df[['Alternative Name', 'Web address', 'Local Authority', 'Area', 'ACE Region', 'ONS Region', 'Funding Source (GIA or LOT)', 'Funding Source Description', 'Funding Band Description', 'Discipline', 'Portfolio funded in 2015-18?', 'Bridge Organisation', 'Technical Uplift', 'Technical Reduction', 'Museum Development']].copy()

In [10]:
# Export files

# separate csv
df.to_csv(outputCSVAll)
dfPortfolio.to_csv(outputCSVPortfolio)
dfFunding.to_csv(outputCSVFunding)

# some folks like Excel!
writer = pd.ExcelWriter(outputXLS)
df.to_excel(writer, 'All')
dfPortfolio.to_excel(writer, 'Portfolio')
dfFunding.to_excel(writer, 'Funding')
writer.save()
writer.close()